In [6]:
import pandas as pd

df_source = pd.read_excel('~/Downloads/english_words_12931.xlsx')
df_source['音标美'].fillna('', inplace=True)
df_source['音标英'].fillna('', inplace=True)
df_source.head()

,单词,音标美,音标英,意思
0,a cushion of,,,网络一层
1,a few,,,na.几个；〔反语〕很多；〈俚〉一点点 网络意思：一些；有些；少量
2,a good deal,,,na.“a deal”的变体 网络意思：许多；大量；诸多
3,a great deal,,,na.很多 网络意思：大量；许多；大量的
4,a little,,,n.一点 网络意思：一些；少许；有点


In [11]:
from pymongo import MongoClient

CONNECTION_STRING = 'mongodb://localhost:27017/quiz-book'
BATCH_INSERT_COUNT = 30

client = MongoClient(CONNECTION_STRING)

english_words_v1 = client['quiz-book']['english_words_v1']

records_to_insert = []
for i, row in df_source.iterrows():
    record = {
        'en': row['单词'],
        'phonetic_us': row['音标美'].replace('美', '').strip(),
        'phonetic_br': row['音标英'].replace('英', '').strip(),
        'zh': row['意思']
    }

    records_to_insert.append(record)

    if len(records_to_insert) >= BATCH_INSERT_COUNT:
        english_words_v1.insert_many(records_to_insert)
        records_to_insert = []

if len(records_to_insert) > 0:
    english_words_v1.insert_many(records_to_insert)